In [54]:
import numpy as np
from PGM_PyLib.MRF import RMRFwO as mrf

In [55]:
# Function to generate random probability distributions for bounding boxes
def generate_random_probabilities(num_boxes, num_states):
    """
    Generate random probability distributions for each bounding box.
    
    Parameters:
    - num_boxes: int, number of bounding boxes
    - num_states: int, number of states (e.g., "tree", "glass", "car", "cup")

    Returns:
    - numpy array of shape (num_boxes, num_states), where each row is a
      probability distribution over the states for a bounding box.
    """
    probabilities = np.random.rand(num_boxes, num_states)  # Generate random values
    probabilities /= probabilities.sum(axis=1, keepdims=True)  # Normalize each row to sum to 1
    return probabilities

# Generate the 4x4 relational probability matrix (for the 4 states)
def generate_symmetric_matrix(n):
    """Generate an n x n symmetric matrix for 0-based indexing."""
    mat = np.random.rand(n, n)  # Generate random values
    mat = (mat + mat.T) / 2  # Make the matrix symmetric
    np.fill_diagonal(mat, 1)  # Fill the diagonal with 1s for self-relations
    return mat

# Define psi and Uf functions
def psi(state_i, state_j):
    """
    Return the probability of transitioning from state_i to state_j
    based on the relational probability matrix (0-based indexing).
    """
    return relational_probabilities[state_i, state_j]

def Uf(rmrf, observation, row, col):
    """
    Local energy function for a bounding box (3 rows, 1 column RMRF).
    
    This function calculates the energy based on the current state of the bounding box
    and its neighboring bounding boxes (if any).
    
    Parameters:
    - rmrf: 2D numpy array, current RMRF matrix (3 rows, 1 column, states should be 0-based)
    - observation: bounding box probability distributions
    - row: int, row index (0-based, corresponding to the bounding box)
    - col: int, column index (should always be 0 since it's a 1-column matrix)

    Returns:
    - float, the calculated energy for the bounding box at row `row`
    """
    current_state = rmrf[row, col]
    energy = 0.0
    
    # Get the observation probability for the current bounding box and state
    obs_prob = observation[row][current_state]  # Use the row index to access bounding box probabilities
    energy -= np.log(obs_prob)  # Factor in the observation likelihood

    # Consider relationships with neighboring bounding boxes (if relevant)
    if row > 0:  # Relationship with the previous bounding box
        prev_state = rmrf[row - 1, col]
        prob = psi(current_state, prev_state)  # Access relational probabilities
        energy -= np.log(prob)  # Energy is -log(psi)
    if row < len(rmrf) - 1:  # Relationship with the next bounding box
        next_state = rmrf[row + 1, col]
        prob = psi(current_state, next_state)  # Access relational probabilities
        energy -= np.log(prob)  # Energy is -log(psi)
    
    return energy

In [56]:
# Define the states
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros((3, 1), dtype=int)  # Initial RMRF values (three rows, one column)
print("Initial RMRF\n", r)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
relational_probabilities = generate_symmetric_matrix(4)
print("\nRandom Relational Probability Matrix (ConceptNet):")
print(relational_probabilities)

# Generate random probability distributions for 3 bounding boxes and 4 states (this should come from the Computer Vision observation)
bounding_box_probabilities = generate_random_probabilities(num_boxes=3, num_states=4)
print("\nRandom Bounding Box Probabilities (Computer Vision):")
print(bounding_box_probabilities, "\n")

# Create an instance of RMRFwO and run inference
mr = mrf(s, r, bounding_box_probabilities)  # Use the bounding box probabilities as the observation

# Run inference using the Uf function
result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")

print("\nFinal RMRF after inference (Bounding Box States):")
print(result)  # This will be a 3x1 matrix with the final state for each bounding box

Initial RMRF
 [[0]
 [0]
 [0]]

Random Relational Probability Matrix (ConceptNet):
[[1.         0.48265977 0.38697328 0.70314562]
 [0.48265977 1.         0.66932358 0.41594745]
 [0.38697328 0.66932358 1.         0.69400817]
 [0.70314562 0.41594745 0.69400817 1.        ]]

Random Bounding Box Probabilities (Computer Vision):
[[0.19736442 0.16445522 0.238936   0.39924436]
 [0.24669894 0.26707415 0.41330297 0.07292394]
 [0.22859731 0.03363544 0.54743894 0.1903283 ]] 

Succesfully finish, iteration: 1

Final RMRF after inference (Bounding Box States):
[[3]
 [0]
 [0]]
